# 🧪 Hands-On Lab: Multi-Agent Workflow with LangChain in Databricks

**Using Databricks Foundation Models (FREE) or OpenAI**

---

### 📍 Scenario

You are working for an **auto insurance company** that processes thousands of claims submitted as free-form text. The business goal is to transform these unstructured claim descriptions into **consistent, machine-actionable decisions** while maintaining traceability and control over each step of the reasoning process.

Instead of relying on a single monolithic prompt, you will design a **multi-stage generative AI workflow** composed of specialized, task-aligned components. Each stage is responsible for a clearly defined task and produces structured output that can be validated and consumed downstream.

---

#### The workflow is composed of the following logical stages:

1. **Extraction Stage**
   A structured prompt extracts key fields—such as claim type, incident description, and severity—from a raw claim record stored in a Databricks table.

2. **Policy Validation Stage**
   A validation step determines whether the associated policy is active and eligible for coverage based on structured inputs.

3. **Assessment Stage**
   An LLM-driven reasoning step evaluates the extracted claim data and policy status to determine whether the claim should be auto-approved or flagged for manual review.

4. **Resolution Stage**
   A final structured response consolidates the outputs of previous stages into a machine-readable decision record suitable for downstream systems.

---

This design mirrors **real-world enterprise pipelines** where prompt-task alignment, structured outputs, tool ordering, and modular composition are essential for reliability, auditability, and scalability.

---

### 🎯 Objective

By the end of this lab, you will be able to:

- **Apply prompt-task alignment** to ensure each stage of a workflow performs the correct LLM task (extraction, classification, or transformation).

- **Design structured prompts** that produce consistent, machine-readable outputs suitable for automated pipelines.

- **Translate a business use case** into a multi-stage AI pipeline with clearly defined inputs and outputs.

- **Define and order reasoning steps** to ensure downstream components receive the correct data at the correct time.

- **Use LangChain abstractions** to compose prompts, models, and reasoning stages without relying on monolithic prompts.

- **Execute the workflow in Databricks**, using PySpark for data preparation and OpenAI models for structured reasoning.

- **Observe how modular design** improves interpretability, debuggability, and control in generative AI systems.

You will simulate the complete workflow inside a Databricks notebook, focusing on **design correctness** rather than optimization, to reinforce the architectural principles introduced in Chapter 2.

---


### 📋 Prerequisites

Before running this lab, ensure you have:

#### **1. Databricks Environment**
- A Databricks Workspace (Community Edition or paid tier)
- Access to Foundation Model Serving Endpoints (available in most Databricks workspaces)

#### **2. Model Endpoint Configuration**

This notebook uses **Databricks Foundation Models** which are FREE and don't require API keys.

**To configure your model endpoint:**

1. In your Databricks workspace, navigate to: **Serving** → **Serving Endpoints**
2. Look for available Foundation Model endpoints, such as:
   - `databricks-meta-llama-3-3-70b-instruct` (recommended)
   - `databricks-meta-llama-3-1-70b-instruct`
   - `databricks-dbrx-instruct`
   - `databricks-mixtral-8x7b-instruct`

3. **Copy the endpoint name** (just the name, NOT the full URL)

**The notebook is pre-configured with:** `databricks-meta-llama-3-3-70b-instruct`

- ✅ If this model is available in your workspace: No changes needed!
- ⚙️ If you need a different model: Update the endpoint name in **Step 5** below

**Important:** Use only the endpoint **name** in your code:
```python
# ✅ CORRECT
endpoint="databricks-meta-llama-3-3-70b-instruct"

# ❌ WRONG - Don't use the full URL
endpoint="https://adb-1234567890.15.azuredatabricks.net/..."
```

Databricks automatically resolves the endpoint name to the correct URL when running in your workspace.

#### **3. Authentication**

When running in Databricks:
- ✅ No API keys needed - Databricks uses your workspace session
- ✅ No URLs needed - Just the endpoint name
- ✅ Completely FREE - Foundation Models are included

#### **4. Alternative Model Options (Optional)**

If you prefer to use other models instead of Databricks Foundation Models:
- **OpenAI**: Requires API key and billing setup
- **Azure OpenAI**: Requires Azure subscription
- **Local Ollama**: Requires local installation

See the model configuration section in **Step 5** for details.

---


### 🔧 Step 1: Install Required Packages

To work with LangChain agents in a modern, non-deprecated way, you need to install the latest versions of the required libraries.

**What's being installed:**

- `langchain-openai`: The modern package for OpenAI integration with LangChain (replaces deprecated `langchain.llms.OpenAI`)
- `langchain`: Core LangChain framework for building agent workflows (v1.0+)
- `langchain-community`: Community-contributed tools and integrations (includes Databricks models)
- `langchain-core`: Core abstractions including prompts, chains, and runnables
- `langgraph`: Modern agent framework (required for `create_agent`)
- `openai`: Official OpenAI Python client

This ensures you're using the **current, supported APIs** without any deprecated imports.

> ⚠️ **Note**: After installation, you'll need to restart the Python kernel to ensure the new packages are loaded properly.

---

#### 💡 **Model Options Available**

After installing these packages, you can use:
- ✅ **OpenAI models** (requires API key and billing)
- ✅ **Databricks Foundation Models** (FREE for Databricks users)
- ✅ **Azure OpenAI** (for enterprise users)
- ✅ **Local models via Ollama** (completely free)


In [0]:

%pip install --upgrade langchain-openai langchain langchain-community langchain-core langgraph openai



### 🔄 Step 2: Restart the Python Kernel

After installing or upgrading packages in Databricks, it's important to restart the Python runtime so your notebook picks up the new dependencies cleanly.

This ensures that:
- All newly installed packages are available in the Python environment
- No conflicts exist between old and new package versions
- Import statements will reference the correct module versions


In [0]:
%restart_python

### 🔑 Step 3: API Key Setup (OPTIONAL - Skip if Using Databricks Models)

> ⚡ **Quick Note**: If you're using **Databricks Foundation Models** (the default in this notebook), you can **SKIP this entire step**. No API key needed!

---

#### 🎯 When Do You Need This Step?

**Skip this step if:**
- ✅ You're using Databricks Foundation Models (default option)
- ✅ You want a completely FREE experience

**Complete this step only if:**
- ❌ You want to use OpenAI models (requires billing)
- ❌ You want to use Azure OpenAI

---

#### 📌 How to Get Your OpenAI API Key (Optional)

1. Go to [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)
2. Log in or create an OpenAI account
3. Click **"Create new secret key"**
4. Copy the generated key (it starts with `sk-...`)
5. Keep it safe — you won't see it again!

---

#### 🔐 How to Set the Key in Databricks

You have **two options** for setting your API key:

---

##### **Option 1: Quick Setup (Development/Learning)**

Paste your key directly in the cell below using `os.environ`:

```python
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-..."  # Replace with your actual key
```

> ⚠️ **Important**: This method is for development and learning purposes only. Never commit API keys to version control!

---

##### **Option 2: Secure Setup (Production/Best Practice)**

Use **Databricks Secrets** to securely store your API key:

**Step 1: Create a secret scope (one-time setup)**
```bash
# In Databricks CLI or notebook
databricks secrets create-scope --scope my-secrets
```

**Step 2: Store your API key**
```bash
databricks secrets put --scope my-secrets --key openai-api-key
# This will open an editor where you paste your key
```

**Step 3: Retrieve the key in your notebook**
```python
import os
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="my-secrets", key="openai-api-key")
```

---

#### 🔧 **Choose Your Method Below**


In [0]:
import os

# ⚡ SKIP THIS CELL if you're using Databricks Foundation Models (the default)

# OPTION 1: Direct setup (for learning/development with OpenAI)
# Uncomment the line below and add your key if you want to use OpenAI:
# os.environ["OPENAI_API_KEY"] = "sk-..."  # ⚠️ Replace with your actual key

# OPTION 2: Secure setup (for production with OpenAI)
# Uncomment the line below:
# os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="my-secrets", key="openai-api-key")

# Verify the key is set (optional)
if os.environ.get("OPENAI_API_KEY", "").startswith("sk-"):
    print("✅ OpenAI API key is set correctly")
else:
    print("ℹ️  No OpenAI API key set - that's OK if you're using Databricks Foundation Models!")
    print("   (This is the default and recommended option for this lab)")


### 🗃️ Step 4: Load Sample Claims Data into a Spark Table

This step creates a simulated insurance claims dataset using PySpark. This represents the **raw, unstructured data** that your multi-agent workflow will process.

---

#### 📊 Dataset Structure

The dataset includes the following fields:

- **`claim_id`**: Unique identifier for each claim (e.g., C101, C102)
- **`claimant_name`**: Name of the person who filed the claim
- **`damage_description`**: Free-form text describing the incident (unstructured input)
- **`estimated_damage`**: Approximate repair cost in dollars
- **`policy_id`**: Reference to the insurance policy number

---

#### 🎯 Why This Matters

In real-world scenarios, claims arrive as unstructured text from various sources (emails, forms, mobile apps). Your AI pipeline must:

1. **Extract** structured information from this raw text
2. **Validate** that the policy is active
3. **Assess** whether the claim meets auto-approval criteria
4. **Resolve** the claim with a final decision

Once loaded, the data is registered as a temporary SQL view called `claims`, which can be queried by your agents during the workflow.

> 🧪 This simulates a production data lake or warehouse table that would feed into your AI pipeline.


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Note: In Databricks, the 'spark' session is pre-initialized and ready to use
# No need to create a SparkSession manually

# Define schema and sample data
schema = StructType([
    StructField("claim_id", StringType(), True),
    StructField("claimant_name", StringType(), True),
    StructField("damage_description", StringType(), True),
    StructField("estimated_damage", DoubleType(), True),
    StructField("policy_id", StringType(), True)
])

data = [
    ("C101", "Alice Jones", "Rear-end collision, bumper damage", 2200.0, "P1001"),
    ("C102", "David Kim", "Broken windshield and headlight", 850.0, "P1002"),
    ("C103", "Maria Patel", "Side door dent and paint scratch", 1200.0, "P1003")
]

df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("claims")

display(df)


### 🧠 Step 5: Initialize the LLM (Multiple Options)

In this step, you'll initialize a **language model** using the modern, non-deprecated API. You have several options depending on your needs and budget.

---

#### 🔄 What Changed from Older Versions?

**Old (Deprecated) Way:**
```python
from langchain.llms import OpenAI  # ❌ Deprecated
llm = OpenAI(temperature=0)
```

**New (Current) Way:**
```python
from langchain_openai import ChatOpenAI  # ✅ Modern
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
```

---

#### 🎯 Model Options

Choose one of the following options based on your situation:

---

##### **Option 1: OpenAI (Recommended for Production)**

**Requirements**:
- Valid OpenAI API key
- Active billing account with available credits

**Pros**:
- Best performance and reliability
- Excellent function calling support
- Industry standard

**Cons**:
- Requires paid account
- Usage-based pricing

```python
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
```

---

##### **Option 2: Databricks Foundation Models (Free for Databricks Users)**

**Requirements**:
- Databricks workspace with Foundation Model APIs enabled

**Pros**:
- ✅ **FREE** for Databricks users
- No external API key needed
- Integrated with Databricks security

**Cons**:
- Only available in Databricks environment
- May have different capabilities than OpenAI

```python
from langchain_community.chat_models import ChatDatabricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0
)
```

---

##### **Option 3: Azure OpenAI (Enterprise)**

**Requirements**:
- Azure subscription
- Azure OpenAI resource deployed

**Pros**:
- Enterprise-grade security and compliance
- Data residency control
- SLA guarantees

**Cons**:
- Requires Azure setup
- More complex configuration

```python
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_endpoint="https://your-resource.openai.azure.com/",
    api_key="your-azure-key",
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo",
    temperature=0
)
```

---

##### **Option 4: Local Models with Ollama (Completely Free)**

**Requirements**:
- Ollama installed locally or on cluster
- Sufficient compute resources

**Pros**:
- ✅ **100% FREE**
- No API keys needed
- Complete data privacy
- No rate limits

**Cons**:
- Requires local setup
- May have lower quality outputs
- Slower inference

```python
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(
    model="llama3",
    temperature=0
)
```

---

#### ⚙️ Configuration Parameters

- **`model`**: Specifies which model to use
- **`temperature=0`**: Makes outputs deterministic (same input = same output), which is critical for production pipelines

> 💡 **Best Practice**: Always use `temperature=0` for business logic and decision-making tasks to ensure consistency and reliability.

---

#### 🚨 **Troubleshooting OpenAI Errors**

If you see errors like:
- **`AuthenticationError (401)`**: Your API key is invalid or not set correctly
- **`RateLimitError (429)`**: You've exceeded your quota or don't have billing set up

**Solutions**:
1. Check your OpenAI billing at: https://platform.openai.com/account/billing
2. Add credits to your account
3. Or switch to **Option 2 (Databricks Foundation Models)** which is FREE

---

#### 🔧 **Choose Your Model Below**

> 📌 **IMPORTANT**: If you're using Databricks Foundation Models (default), make sure the endpoint name below matches what's available in YOUR workspace. See the **"Databricks Workspace Setup"** section at the top of this notebook for instructions.


In [0]:
from langchain_community.chat_models import ChatDatabricks

# OPTION 2: Databricks Foundation Models (FREE - RECOMMENDED) ⭐
# This is the default option - no API keys or billing required!
#
# ⚠️ CONFIGURATION: Update the endpoint name if needed
# - Go to your Databricks workspace: Serving → Serving Endpoints
# - Find an available Foundation Model endpoint
# - Replace the endpoint name below with YOUR endpoint name (just the name, NOT the full URL)
#
# ✅ CORRECT: endpoint="databricks-meta-llama-3-3-70b-instruct"
# ❌ WRONG:   endpoint="https://adb-123456.azuredatabricks.net/..."
#
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",  # ← Change this if needed
    temperature=0
)

# OPTION 1: OpenAI (requires valid API key and billing)
# Uncomment the lines below to use OpenAI instead:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# OPTION 3: Azure OpenAI (for enterprise users)
# Uncomment and configure the lines below:
# from langchain_openai import AzureChatOpenAI
# llm = AzureChatOpenAI(
#     azure_endpoint="https://your-resource.openai.azure.com/",
#     api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
#     api_version="2024-02-15-preview",
#     deployment_name="gpt-35-turbo",
#     temperature=0
# )

# OPTION 4: Local Ollama (completely free, requires Ollama installed)
# Uncomment the lines below:
# from langchain_community.chat_models import ChatOllama
# llm = ChatOllama(model="llama3", temperature=0)

print(f"✅ LLM initialized: {llm.__class__.__name__}")
print(f"✅ Using Databricks Foundation Model: databricks-meta-llama-3-3-70b-instruct")
print(f"✅ Cost: FREE (included with Databricks)")
print(f"✅ No API keys required!")


### 📝 Step 6: Create Structured Prompts for Each Workflow Stage

In this step, you'll define **prompt templates** for each of the four stages in your multi-agent workflow. Each prompt is designed to perform a specific task and produce structured, machine-readable output.

---

#### 🎯 The Four Stages

1. **Extraction Stage**: Extract structured fields from raw claim data
2. **Policy Validation Stage**: Determine if a policy is active and eligible
3. **Assessment Stage**: Decide if the claim should be auto-approved or manually reviewed
4. **Resolution Stage**: Generate the final decision record

---

#### 🔍 Why Structured Prompts Matter

- **Consistency**: Each stage produces predictable output formats
- **Modularity**: Stages can be tested, debugged, and improved independently
- **Traceability**: You can audit each step of the decision-making process
- **Downstream Integration**: Structured outputs can be consumed by databases, APIs, or other systems

---

#### 📐 Prompt Design Principles

Each prompt follows these best practices:

- **Clear task definition**: Tells the LLM exactly what to do
- **Structured output format**: Specifies how the response should be formatted
- **Minimal ambiguity**: Uses precise language to reduce variability
- **Context-appropriate**: Tailored to the specific stage's responsibility

> 🧩 This is a core principle of **prompt-task alignment**: matching the right prompt structure to the right type of task.

---

#### 📦 Modern Import Structure

**Important**: We're using `langchain_core.prompts` instead of the older `langchain.prompts`:

```python
from langchain_core.prompts import PromptTemplate  # ✅ Modern
```

This is part of LangChain's modular architecture where core abstractions live in `langchain-core`.


In [0]:
from langchain_core.prompts import PromptTemplate

# Stage 1: Extraction Prompt
extraction_prompt = PromptTemplate.from_template(
    """You are an insurance claim data extraction specialist.

Extract the following structured information from this claim:

Claim Data: {claim_data}

Provide the output in this exact format:
- Claim ID: [value]
- Claimant Name: [value]
- Incident Type: [classify as: collision, vandalism, weather, or other]
- Damage Description: [value]
- Estimated Cost: [value]
- Policy ID: [value]
- Severity: [classify as: low, medium, or high based on cost]

Be precise and use only the information provided."""
)

# Stage 2: Policy Validation Prompt
validation_prompt = PromptTemplate.from_template(
    """You are a policy validation specialist.

Policy ID: {policy_id}
Policy Status: {policy_status}

Determine if this policy is eligible for claim processing.

Provide the output in this exact format:
- Policy ID: [value]
- Status: [Active/Inactive]
- Eligible for Coverage: [Yes/No]
- Reason: [brief explanation]"""
)

# Stage 3: Assessment Prompt
assessment_prompt = PromptTemplate.from_template(
    """You are a claim assessment specialist.

Claim Information:
{claim_info}

Policy Status:
{policy_status}

Based on the following rules:
1. If estimated cost > $2000, flag for manual review
2. If policy is not active, reject automatically
3. If cost <= $2000 and policy is active, auto-approve

Provide the output in this exact format:
- Decision: [Auto-Approve/Manual Review/Reject]
- Reason: [brief explanation]
- Estimated Cost: [value]
- Risk Level: [Low/Medium/High]"""
)

# Stage 4: Resolution Prompt
resolution_prompt = PromptTemplate.from_template(
    """You are a claim resolution specialist.

Assessment Result:
{assessment}

Generate a final decision record in this exact format:
- Final Decision: [Approved/Pending Review/Rejected]
- Next Steps: [specific actions required]
- Processing Status: [Complete/Requires Human Review]
- Timestamp: [current stage]"""
)


### 🛠️ Step 7: Define Tool Functions for Multi-Agent Workflow

This step defines four custom Python functions that act as **tools** for your LangChain agent. Each tool corresponds to one of the four workflow stages and encapsulates the business logic for that stage.

---

#### 🔧 Tool 1: Extract Claim Details (Extraction Stage)

**Purpose**: Query the Spark table and retrieve raw claim data

**Input**: `claim_id` (string)

**Output**: Formatted string with claim details

**Business Logic**:
- Queries the `claims` table using Spark SQL
- Returns structured claim information if found
- Returns error message if claim ID doesn't exist

---

#### 🔧 Tool 2: Validate Policy (Policy Validation Stage)

**Purpose**: Check if a policy is active and eligible for coverage

**Input**: `policy_id` (string)

**Output**: Policy validation status

**Business Logic**:
- Simulates a policy database lookup
- Checks against a list of valid policy IDs
- Returns validation result with eligibility status

> 📝 **Note**: In production, this would query a real policy management system or database.

---

#### 🔧 Tool 3: Assess Damage with LLM (Assessment Stage)

**Purpose**: Use the LLM to evaluate claim data and make an approval decision

**Input**: Claim information text

**Output**: Assessment decision (Auto-Approve/Manual Review/Reject)

**Business Logic**:
- Extracts the dollar amount from the claim text
- Invokes the LLM with the assessment prompt
- Returns structured decision based on business rules

---

#### 🔧 Tool 4: Finalize Resolution (Resolution Stage)

**Purpose**: Generate the final decision record

**Input**: Assessment result

**Output**: Final formatted decision message

**Business Logic**:
- Takes the assessment output
- Formats it into a final resolution message
- Prepares the output for downstream systems

---

> 🧩 These functions will be wrapped as LangChain tools in the next step, allowing a reasoning agent to call them dynamically based on the task prompt.


In [0]:
import re

# Tool 1: Extract Claim Details from Spark Table
def extract_claim_details(claim_id):
    """
    Extraction Stage: Retrieve raw claim data from the Spark table.

    Args:
        claim_id: Unique identifier for the claim

    Returns:
        Formatted string with claim details or error message
    """
    # Use DataFrame API with column-based filtering (Databricks best practice)
    # This avoids SQL injection and is more efficient
    from pyspark.sql import functions as F

    claim_df = spark.table("claims").filter(F.col("claim_id") == claim_id)
    claim = claim_df.first()

    if not claim:
        return f"No claim found for ID {claim_id}"

    # Return structured claim data
    return f"Claimant: {claim.claimant_name}, Damage: {claim.damage_description}, Estimate: ${claim.estimated_damage}, Policy#: {claim.policy_id}"


# Tool 2: Validate Policy Status
def validate_policy(policy_id):
    """
    Policy Validation Stage: Check if a policy is active and eligible.

    Args:
        policy_id: Policy identifier to validate

    Returns:
        Policy validation status message
    """
    # Simulate valid policies (in production, this would query a policy database)
    valid_policies = {"P1001", "P1002", "P1003"}

    if policy_id in valid_policies:
        return f"Policy {policy_id} is valid and active."
    else:
        return f"Policy {policy_id} is NOT valid or inactive."


# Tool 3: Assess Damage Using LLM
def assess_damage_llm(text):
    """
    Assessment Stage: Use LLM to evaluate claim and determine approval decision.

    Args:
        text: Claim information text containing cost estimate

    Returns:
        LLM-generated assessment decision
    """
    # Extract dollar amount from text
    amount_match = re.search(r"\$?(\d+[.,]?\d*)", text)
    if not amount_match:
        return "Could not extract a valid dollar amount from the claim."

    amount = amount_match.group(1)

    # Invoke LLM with assessment prompt
    assessment_result = llm.invoke(
        assessment_prompt.format(
            claim_info=text,
            policy_status="Active"  # This would come from validate_policy in real workflow
        )
    )

    return assessment_result.content


# Tool 4: Finalize Resolution
def finalize_resolution(assessment):
    """
    Resolution Stage: Generate final decision record.

    Args:
        assessment: Assessment result from previous stage

    Returns:
        Final formatted decision message
    """
    return f"Final decision: {assessment}"


### 🤖 Step 8: Register Tools and Initialize a Reasoning Agent

Now that your helper functions are ready, you'll convert them into **LangChain-compatible tools** and initialize a **reasoning agent** using the modern OpenAI API.

---

#### 🔧 What Are LangChain Tools?

Tools are functions that an agent can call to perform specific tasks. Each tool has:

- **Name**: A unique identifier the agent uses to reference the tool
- **Function**: The Python function to execute
- **Description**: Instructions that tell the agent when and how to use the tool

The agent reads these descriptions and decides which tools to call based on the user's request.

---

#### 🧠 Agent Type: `create_agent`

We're using the **modern `create_agent` API** from LangChain v1, which is the current recommended way to build agents.

**How the Agent Works:**

1. **Reason**: The agent analyzes the task and decides what to do
2. **Act**: The agent calls a tool to perform an action
3. **Observe**: The agent examines the tool's output
4. **Repeat**: The agent continues reasoning and acting until the task is complete

This uses the ReAct (Reasoning + Acting) pattern under the hood.

---

#### 🔄 Modern vs. Deprecated Approach

**Old (Deprecated) Way:**
```python
from langchain.agents import initialize_agent, AgentType
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

**New (Current) Way:**
```python
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful insurance claim processing assistant."
)
```

---

#### 🎯 Why This Matters

- **Non-deprecated**: Uses the current LangChain API
- **More flexible**: Allows custom prompts and better control
- **Production-ready**: Follows modern best practices

> 💡 The agent will automatically orchestrate the four-stage workflow by calling the tools in the correct order.


In [0]:
from langchain.agents import create_agent
from langchain_core.tools import tool

# Define tools using the @tool decorator (modern approach)
@tool
def extract_claim(claim_id: str) -> str:
    """Look up claim details from the Spark database. Input should be a claim ID like 'C101'."""
    return extract_claim_details(claim_id)

@tool
def validate_policy_tool(policy_id: str) -> str:
    """Check if a policy is valid and active. Input should be a policy ID like 'P1001'."""
    return validate_policy(policy_id)

@tool
def assess_damage(claim_info: str) -> str:
    """Assess claim damage and determine approval decision. Input should be claim information text with cost estimate."""
    return assess_damage_llm(claim_info)

@tool
def finalize_resolution_tool(assessment: str) -> str:
    """Generate the final claim decision. Input should be the assessment result."""
    return finalize_resolution(assessment)

# Create list of tools
tools = [extract_claim, validate_policy_tool, assess_damage, finalize_resolution_tool]

# Create the agent using the modern create_agent API
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="""You are a helpful insurance claim processing assistant.

Your job is to process insurance claims through a multi-stage workflow:
1. Extract claim details using the claim ID
2. Validate the policy is active
3. Assess the damage and determine if it should be auto-approved or manually reviewed
4. Finalize the resolution with a decision

Always follow these steps in order and provide clear reasoning for your decisions."""
)


### 🚀 Step 9: Execute the Multi-Agent Workflow

Now it's time to run the complete workflow! The agent will process a claim by automatically orchestrating all four stages.

---

#### 🔄 What Happens When You Run This?

When you execute `agent.invoke()`, the agent will:

1. **Read your prompt**: "Process claim C101"
2. **Plan the workflow**: Determine which tools to call and in what order
3. **Execute Stage 1 (Extraction)**: Call `extract_claim` to retrieve claim data
4. **Execute Stage 2 (Validation)**: Call `validate_policy_tool` to check policy status
5. **Execute Stage 3 (Assessment)**: Call `assess_damage` to evaluate the claim
6. **Execute Stage 4 (Resolution)**: Call `finalize_resolution_tool` to generate the final decision
7. **Return the result**: Provide a complete, structured response

---

#### 🧠 Observing the Reasoning Process

The agent will show you its internal reasoning process in the output:

- **Tool Calls**: Which tools the agent decides to use
- **Tool Inputs**: What data it passes to each tool
- **Tool Outputs**: What each tool returns
- **Final Answer**: The complete result

This transparency is crucial for:
- **Debugging**: Understanding why the agent made certain decisions
- **Auditing**: Tracking the decision-making process for compliance
- **Optimization**: Identifying bottlenecks or inefficiencies

---

#### 📊 Expected Output

For claim C101 (Alice Jones, $2200 damage):

- **Extraction**: Successfully retrieves claim details
- **Validation**: Confirms policy P1001 is active
- **Assessment**: Flags for manual review (cost > $2000)
- **Resolution**: Generates final decision record

> 💡 Try running this with different claim IDs (C102, C103) to see how the workflow adapts to different scenarios!


In [0]:
# Run the multi-agent workflow on claim C101
try:
    result = agent.invoke({"messages": [{"role": "user", "content": "Process claim C101"}]})
    print("\n" + "="*60)
    print("FINAL RESULT:")
    print("="*60)
    # Extract the final message from the agent
    final_message = result["messages"][-1]
    print(final_message.content)
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying alternative invocation method...")
    # Alternative method for some model providers
    result = agent.invoke({"messages": [("user", "Process claim C101")]})
    print("\n" + "="*60)
    print("FINAL RESULT:")
    print("="*60)
    final_message = result["messages"][-1]
    print(final_message.content)


### 🔍 Step 10: Test with Additional Claims

Let's test the workflow with the other claims to see how it handles different scenarios.

---

#### 📋 Test Scenarios

**Claim C102** (David Kim):
- Estimated damage: $850
- Expected outcome: Auto-approve (cost < $2000)

**Claim C103** (Maria Patel):
- Estimated damage: $1200
- Expected outcome: Auto-approve (cost < $2000)

---

#### 🎯 What to Observe

Pay attention to how the agent:

1. **Adapts its reasoning** based on different cost amounts
2. **Maintains consistency** in the workflow structure
3. **Produces structured outputs** at each stage
4. **Makes different decisions** based on business rules

This demonstrates the power of **modular, multi-stage workflows** where each component has a clear responsibility.


In [0]:
# Test with claim C102 (lower cost - should auto-approve)
print("\n" + "="*60)
print("TESTING CLAIM C102")
print("="*60)
try:
    result_c102 = agent.invoke({"messages": [{"role": "user", "content": "Process claim C102"}]})
except:
    result_c102 = agent.invoke({"messages": [("user", "Process claim C102")]})
print("\n" + "="*60)
print("FINAL RESULT FOR C102:")
print("="*60)
print(result_c102["messages"][-1].content)


In [0]:
# Test with claim C103 (medium cost - should auto-approve)
print("\n" + "="*60)
print("TESTING CLAIM C103")
print("="*60)
try:
    result_c103 = agent.invoke({"messages": [{"role": "user", "content": "Process claim C103"}]})
except:
    result_c103 = agent.invoke({"messages": [("user", "Process claim C103")]})
print("\n" + "="*60)
print("FINAL RESULT FOR C103:")
print("="*60)
print(result_c103["messages"][-1].content)


### ✅ Understanding the Multi-Agent Workflow Output

Congratulations! You've successfully built and executed a **multi-stage generative AI workflow** for insurance claim processing.

---

#### 🎯 Key Takeaways

**1. Prompt-Task Alignment**
- Each stage had a specific prompt designed for its task (extraction, validation, assessment, resolution)
- This ensures the LLM performs the correct type of reasoning at each step

**2. Structured Outputs**
- Every stage produced machine-readable, consistent outputs
- These outputs can be validated, logged, and consumed by downstream systems

**3. Modular Design**
- Each tool is independent and can be tested separately
- Changes to one stage don't break the entire workflow
- Easy to add new stages or modify existing ones

**4. Tool Ordering**
- The agent automatically determined the correct sequence of operations
- Data flows logically from extraction → validation → assessment → resolution

**5. Observability**
- The verbose output shows every step of the reasoning process
- This is essential for debugging, auditing, and compliance

---

#### 🏢 Real-World Applications

This pattern applies to many enterprise scenarios:

- **Financial Services**: Loan application processing, fraud detection
- **Healthcare**: Medical claim adjudication, patient triage
- **Customer Service**: Ticket routing, automated responses
- **Legal**: Contract analysis, compliance checking
- **HR**: Resume screening, candidate evaluation

---

#### 🚀 Next Steps

To extend this lab, you could:

1. **Add more validation rules** (e.g., check claim history, verify claimant identity)
2. **Implement structured output parsing** using Pydantic models
3. **Add error handling** for edge cases and invalid inputs
4. **Integrate with real databases** instead of simulated data
5. **Add logging and monitoring** for production deployment
6. **Implement human-in-the-loop** for manual review cases
7. **Create unit tests** for each tool function

---

#### 📚 What You've Learned

By completing this lab, you've demonstrated:

✅ How to design multi-stage AI workflows with clear task boundaries
✅ How to use modern LangChain APIs without deprecated code
✅ How to create structured prompts for consistent outputs
✅ How to orchestrate multiple tools using a reasoning agent
✅ How to execute AI workflows in Databricks with PySpark integration
✅ How modular design improves reliability and maintainability

---

> 🎓 **Certification Tip**: The concepts in this lab—prompt-task alignment, structured outputs, tool composition, and modular workflows—are core topics in the Databricks Generative AI Engineer Associate certification exam.


### 🎓 Summary: Architecture Principles Demonstrated

This lab reinforced the following architectural principles from Chapter 2:

---

#### 1️⃣ **Separation of Concerns**
Each stage has a single, well-defined responsibility:
- Extraction: Parse raw data
- Validation: Check eligibility
- Assessment: Make decisions
- Resolution: Format outputs

---

#### 2️⃣ **Composability**
Tools can be combined in different ways:
- Add new tools without changing existing ones
- Reorder stages for different workflows
- Reuse tools across multiple agents

---

#### 3️⃣ **Testability**
Each component can be tested independently:
- Unit test individual tool functions
- Integration test the full workflow
- Mock external dependencies (databases, APIs)

---

#### 4️⃣ **Observability**
The workflow provides full visibility:
- Verbose logging shows reasoning steps
- Structured outputs enable monitoring
- Clear error messages aid debugging

---

#### 5️⃣ **Scalability**
The design supports production deployment:
- Stateless tools can run in parallel
- Modular architecture enables horizontal scaling
- Clear interfaces support microservices architecture

---

> 🏆 **Well done!** You've completed the hands-on lab for Chapter 2: Multi-Agent Workflow with LangChain + OpenAI in Databricks.


### 🔧 Troubleshooting Common Issues

If you encounter errors while running this notebook, here are solutions to common problems:

---

#### ❌ **Error: `Endpoint not found` or `Model endpoint does not exist`**

**Problem**: The Databricks model endpoint name doesn't match what's available in your workspace

**Solutions**:
1. **Check available endpoints in your workspace**:
   - Navigate to: **Serving** → **Serving Endpoints** in Databricks
   - Look for Foundation Model endpoints (e.g., `databricks-meta-llama-3-3-70b-instruct`)
   - Copy the exact endpoint name

2. **Update the endpoint in Step 5**:
   ```python
   llm = ChatDatabricks(
       endpoint="YOUR-ENDPOINT-NAME-HERE",  # ← Paste your endpoint name
       temperature=0
   )
   ```

3. **Common endpoint names to try**:
   - `databricks-meta-llama-3-3-70b-instruct` (recommended)
   - `databricks-meta-llama-3-1-70b-instruct`
   - `databricks-dbrx-instruct`
   - `databricks-mixtral-8x7b-instruct`

4. **Make sure you're using the endpoint NAME, not the URL**:
   - ✅ CORRECT: `endpoint="databricks-meta-llama-3-3-70b-instruct"`
   - ❌ WRONG: `endpoint="https://adb-123456.azuredatabricks.net/..."`

---

#### ❌ **Error: `AuthenticationError: Error code: 401`**

**Problem**: Invalid or missing OpenAI API key

**Solutions**:
1. **Check your API key is set correctly**:
   ```python
   import os
   print(os.environ.get("OPENAI_API_KEY", "NOT SET")[:10] + "...")
   ```
   Should show: `sk-proj-...` or `sk-...`

2. **Verify your key is valid** at: https://platform.openai.com/account/api-keys

3. **Switch to Databricks Foundation Models (FREE)**:
   ```python
   from langchain_community.chat_models import ChatDatabricks
   llm = ChatDatabricks(
       endpoint="databricks-meta-llama-3-3-70b-instruct",
       temperature=0
   )
   ```

---

#### ❌ **Error: `RateLimitError: Error code: 429`**

**Problem**: OpenAI quota exceeded or no billing set up

**Solutions**:
1. **Check your OpenAI billing**: https://platform.openai.com/account/billing
   - Add credits to your account
   - Verify you have an active payment method

2. **Use Databricks Foundation Models instead (FREE)**:
   ```python
   from langchain_community.chat_models import ChatDatabricks
   llm = ChatDatabricks(
       endpoint="databricks-meta-llama-3-3-70b-instruct",
       temperature=0
   )
   ```
   Then re-run the agent creation cell and execution cells.

3. **Use a local model with Ollama (FREE)**:
   ```bash
   # Install Ollama first: https://ollama.ai
   ollama pull llama3
   ```
   ```python
   from langchain_community.chat_models import ChatOllama
   llm = ChatOllama(model="llama3", temperature=0)
   ```

---

#### ❌ **Error: `ModuleNotFoundError: No module named 'langchain_openai'`**

**Problem**: Packages not installed or kernel not restarted

**Solutions**:
1. **Re-run the installation cell**:
   ```python
   %pip install --upgrade langchain-openai langchain langchain-community langchain-core langgraph openai
   ```

2. **Restart the Python kernel**:
   ```python
   %restart_python
   ```

3. **Verify installation**:
   ```python
   import langchain
   import langchain_openai
   print(f"LangChain version: {langchain.__version__}")
   ```

---

#### ❌ **Error: `ImportError: cannot import name 'create_agent'`**

**Problem**: Old version of LangChain installed

**Solutions**:
1. **Upgrade to LangChain 1.0+**:
   ```python
   %pip install --upgrade langchain>=1.0.0
   %restart_python
   ```

2. **Verify version**:
   ```python
   import langchain
   print(langchain.__version__)  # Should be 1.0.0 or higher
   ```

---

#### ❌ **Error: `AnalysisException: Table or view not found: claims`**

**Problem**: The claims table wasn't created

**Solutions**:
1. **Re-run the data loading cell** (Step 4):
   ```python
   df = spark.createDataFrame(data, schema)
   df.createOrReplaceTempView("claims")
   ```

2. **Verify the table exists**:
   ```python
   spark.sql("SELECT * FROM claims").show()
   ```

---

#### ❌ **Error: Agent produces incorrect or incomplete results**

**Problem**: Model quality or prompt issues

**Solutions**:
1. **Try a more capable model**:
   ```python
   llm = ChatOpenAI(model="gpt-4", temperature=0)  # More expensive but better
   ```

2. **Check tool descriptions are clear**:
   - Each `@tool` function should have a clear docstring
   - The system prompt should be specific

3. **Add more examples to prompts**:
   - Include few-shot examples in your prompt templates

---

#### ❌ **Error: `dbutils is not defined` or Running Outside Databricks**

**Problem**: You're trying to run this notebook outside of a Databricks environment

**Solutions**:

**Option 1: Use OpenAI instead (requires API key)**
```python
import os
from langchain_openai import ChatOpenAI

# Set your API key
os.environ["OPENAI_API_KEY"] = "sk-..."  # Your OpenAI API key

# Use OpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
```

**Option 2: Use local Ollama (completely free)**
```bash
# Install Ollama first: https://ollama.ai
ollama pull llama3
```
```python
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3", temperature=0)
```

**Option 3: Run in Databricks (recommended for this lab)**
- Upload this notebook to your Databricks workspace
- Databricks Community Edition is FREE: https://databricks.com/try-databricks

---

#### 💡 **Best Practices for Success**

1. ✅ **Always restart the kernel** after installing packages
2. ✅ **Run cells in order** from top to bottom
3. ✅ **Verify your model endpoint** matches what's available in your workspace
4. ✅ **Use endpoint NAMES, not URLs** in your code
5. ✅ **Use Databricks Foundation Models** if you don't have OpenAI credits
6. ✅ **Read error messages carefully** - they usually tell you exactly what's wrong

---

#### 🆘 **Still Having Issues?**

If you're still stuck:
1. Check the [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
2. Review the [Databricks Documentation](https://docs.databricks.com/)
3. Search for your error message on [Stack Overflow](https://stackoverflow.com/questions/tagged/langchain)
4. Ask in the [LangChain Discord](https://discord.gg/langchain)


